In [219]:
import pandas as pd


In [220]:
train_data = pd.read_csv('BBC News Train.csv')
test_data = pd.read_csv('BBC News Test.csv')

In [221]:
train_data.head()

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business


In [222]:
#print(train_data.head)
print(train_data.info())
train_data['Category'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ArticleId  1490 non-null   int64 
 1   Text       1490 non-null   object
 2   Category   1490 non-null   object
dtypes: int64(1), object(2)
memory usage: 35.1+ KB
None


Category
sport            346
business         336
politics         274
entertainment    273
tech             261
Name: count, dtype: int64

In [223]:
test_data.head()

,ArticleId,Text
0,1018,qpr keeper day heads for preston queens park r...
1,1319,software watching while you work software that...
2,1138,d arcy injury adds to ireland woe gordon d arc...
3,459,india s reliance family feud heats up the ongo...
4,1020,boro suffer morrison injury blow middlesbrough...


In [224]:
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ArticleId  735 non-null    int64 
 1   Text       735 non-null    object
dtypes: int64(1), object(1)
memory usage: 11.6+ KB
None


In [225]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from numpy import dot
from numpy.linalg import norm
from collections import Counter


In [226]:
print(train_data.Category.values[0])

business


In [232]:
def predictLabels(test_data):
    np.random.seed(0)

    ## Constants
    MAX_FEATURES = 4000
    NGRAM_MIN = 2
    NGRAM_MAX = 3
    MIN_DF = 10
    K_NEIGH = 6
    NUM_HYPER_PLANES = 5

    ##train tfIdf vector
    vectorizer = TfidfVectorizer(ngram_range=(NGRAM_MIN, NGRAM_MAX), max_features=MAX_FEATURES, min_df=MIN_DF)
    train_tfIdfVector = vectorizer.fit_transform(train_data.Text)

    #print(train_tfIdfVector.shape)

    ##random hyper planes
    random_hyper_planes = []
    for i in range(NUM_HYPER_PLANES):
        random_hyper_planes.append(np.random.normal(0, 1, train_tfIdfVector.shape[1]))

    ## bins to store the points with same hash value
    bins = dict()
    for i in range(2**NUM_HYPER_PLANES):
        bins[i] = []
    
    ##placing train data points inside bins
    ## looping through each data point and finding on which direction it belongs to
    for i in tqdm(range(train_tfIdfVector.shape[0])):
        bin_cal = 0
        for j in range(len(random_hyper_planes)):
            if dot(random_hyper_planes[j], train_tfIdfVector[i][0].toarray()[0]) > 0.0:
                bin_cal += 2**j
        bins[bin_cal].append(((train_tfIdfVector[i], train_data.Category.values[i])))

    ## test data tf idf vectors
    test_tfIdfVector = vectorizer.transform(test_data.Text)
    
    ## now for each test data point we have to find which bin it belongs to and its k-NN using cos-sim
    pred = []

    for i in tqdm(range(test_tfIdfVector.shape[0])):
        bin_cal = 0
        
        for j in range(len(random_hyper_planes)):
            if dot(random_hyper_planes[j], test_tfIdfVector[i][0].toarray()[0]) > 0.0:
                bin_cal += 2**j
        
        ## getting all neighbours for this bucket from train set
        neighbours = list(bins[bin_cal])
        
        ## now finding cos-sim of all the train points present in this bin....
        cosine_similarity = []
        for j in range(len(neighbours)):
            cos_sim = dot(neighbours[j][0].toarray()[0], test_tfIdfVector[i][0].toarray()[0])/(norm(neighbours[j][0].toarray()[0])*norm(test_tfIdfVector[i][0].toarray()[0]))
            cosine_similarity.append(cos_sim)

        ##noew taking the nearest k neighbours in the same bin
        nearest_k = np.argsort(cosine_similarity)[::-1][:K_NEIGH]
        #print(nearest_k)
        #pred.append(neighbours[k][1] for k in nearest_k)
        for k in nearest_k:
            pred.append(neighbours[k][1])
        #print(pred)

    #print(len(pred))

    class_labels = sorted(list(set(train_data.Category.values)))
    #print(class_labels)

    ##finding the number of votes each class label has got for each test_data point
    test_pred_count = []

    #print(pred)
    temp = []
    ans = []
    for item in pred:
        temp.append(item)
        if len(temp) == K_NEIGH:
            ans.append(temp)
            temp = []

    #print(ans)

    class_label_pred = []

    for eachList in ans:
        counts = Counter(eachList)
        majority_class = max(counts, key=counts.get)
        class_label_pred.append(majority_class)
        
        


    # for i in range(len(pred)):
    #     result = []
    #     for j in class_labels:
    #         result.append(pred[i].count(j))
    #     test_pred_count.append(result)

    # #print(test_pred_count)
    # ##finding max-votes each class label has got for each data point
    # test_pred_count = np.array(test_pred_count)
    # max_vals = np.max(test_pred_count, axis = 1)

    # ##finding max voted class label
    # print(test_pred_count)
    # print(max_vals)
    # class_label_pred = []
    # for x in range(len(test_data)):
    #     print(x)
    #     max_voted_class_arr = []
    #     max_voted_class_label = ""
    #     for i in range(len(test_pred_count)):
            
    #         for j in range(len(class_labels)):
    #             if test_pred_count[i][j] == max_vals[i]:
    #                 max_voted_class_arr.append(class_labels[j])
    #     print(max_voted_class_arr)
    #     class_label_pred.append(max_voted_class_label)
    
    # print(len(class_label_pred))
    return class_label_pred    
    

In [233]:
predicted_lables = predictLabels(test_data)

100%|██████████| 735/735 [00:04<00:00, 170.71it/s]


In [234]:
result_dict = dict()
result_dict = {
    'ArticleId' : test_data.ArticleId,
    'Category' : predicted_lables
}


In [235]:
output_df = pd.DataFrame(result_dict)

In [236]:
output_df.to_csv('BBC News Sample Solution.csv', index=False)